In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [8]:
iris = load_iris()

In [11]:
# 显示其描述
print("【DESCR】\n", iris.DESCR)
# 显示其特征名称
print("【feature_names】\n", iris.feature_names)
# 显示其特征数据
print("【data】\n", iris.data.shape)
print(iris.data[:5])
# 显示其标签数据
print("【target】\n", iris.target.shape)
print(iris.target[:5])

【DESCR】
 .. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ==

In [12]:
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

(120, 4) (120,)
(30, 4) (30,)


In [14]:
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier()

In [15]:
clr.score(X_test, y_test)

0.9666666666666667

In [21]:
X_test[:2]

array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ]])

In [22]:
clr.predict_proba(X_test[:2])

array([[0. , 0. , 1. ],
       [0. , 0.9, 0.1]])

In [17]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("saved_model/rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [22]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("saved_model/rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]
pred_onx

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0], dtype=int64)